## Building A Chatbot
We'll go over an example of how to design and implement an LLM-powered chatbot. This chatbot will be able to have a conversation and remember previous interactions.

Note that this chatbot that we build will only use the language model to have a conversation. There are several other related concepts that you may be looking for:

- Conversational RAG: Enable a chatbot experience over an external source of data
- Agents: Build a chatbot that can take actions

This will cover the basics which will be helpful for those two more advanced topics.

In [1]:
import os
from dotenv import load_dotenv
load_dotenv(r"C:\Projects\.env")
from langchain_groq import ChatGroq
from langchain_core.messages import AIMessage, HumanMessage , SystemMessage
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [2]:
groq_api_key = os.getenv("GROQ_API_KEY")

In [3]:
llm = ChatGroq(
    model="llama-3.3-70b-versatile",
    groq_api_key=groq_api_key,
    max_tokens=150  # ~100 words
)
response = llm.invoke("I am Soumo working as a product analyst in source desk")


In [4]:
response

AIMessage(content="Hello Soumo, it's nice to meet you. As a product analyst at Source Desk, you must be working on analyzing data and insights to inform product development and improvement. What specific areas of product analysis do you focus on, and what tools or methodologies do you use in your work? I'm here to help and chat, so feel free to share more about your role and interests.", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 79, 'prompt_tokens': 47, 'total_tokens': 126, 'completion_time': 0.287272727, 'prompt_time': 0.00233841, 'queue_time': 0.056221769, 'total_time': 0.289611137}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_e91e6fbd56', 'finish_reason': 'stop', 'logprobs': None}, id='run--19ed6ec2-6999-4401-a44d-ea76e5fc5f1f-0', usage_metadata={'input_tokens': 47, 'output_tokens': 79, 'total_tokens': 126})

In [5]:
llm.invoke(    [
        HumanMessage(content="I am Soumo working as a product analyst in source desk"),
        AIMessage(content="Hello Soumo, it's nice to meet you. As a product analyst at Source Desk, you must be working on analyzing data to inform product decisions and drive business growth. What specific areas of product analysis do you focus on, such as customer behavior, market trends, or product optimization? I'm here to help and chat about your work if you'd like."),
        HumanMessage(content="What is My Name ?")
    ]
).content

'Your name is Soumo, and you work as a Product Analyst at Source Desk.'

### Message History
We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!


In [6]:
store = {}
def get_session_history(session_id: str)-> BaseChatMessageHistory: ## functio will be created to get the session history and 
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]
config1 = {"configurable": {"session_id":"s1"}}
config2 = {"configurable": {"session_id":"s2"}}
with_message_history = RunnableWithMessageHistory(llm,get_session_history)

In [7]:
config1 = {"configurable": {"session_id":"s1"}}
with_message_history.invoke(

    [HumanMessage(content="I am Soumo working as a product analyst in source desk"),
     AIMessage(content="Hello Soumo, it's nice to meet you. As a product analyst at Source Desk, you must be working on analyzing data to inform product decisions and drive business growth. What specific areas of product analysis do you focus on, such as customer behavior, market trends, or product optimization? I'm here to help and chat about your work if you'd like."),
     AIMessage(content=" \n\nAlso, I can offer some general information about product analysis if you're interested. Or we could discuss topics like data analysis, product development, or industry trends. Just let me know what's on your mind!"),
     HumanMessage(content="What is My Name ?"),
     AIMessage(content="Your name is Soumo, and you work as a product analyst at Source Desk."),
     HumanMessage(content="Where is SourceDesk located ?")],
    config=config1
).content

"Unfortunately, I don't have that information. You didn't mention the location of Source Desk, and I couldn't find any information about it. Can you please tell me where Source Desk is located?"

In [8]:
list(store.keys())

['s1']

In [9]:
config2 = {"configurable": {"session_id":"s2"}}
with_message_history.invoke(

    [
     HumanMessage(content="I am krish working as a product analyst in source desk"),
     AIMessage(content="Hello Krish, nice to meet you. It sounds like you have an interesting role as a Product Analyst at Source Desk. What specific aspects of product analysis do you focus on, and what are some of the most challenging or rewarding parts of your job?"),
     HumanMessage(content="Source desk is an IT firm, located in kolkata"),
     AIMessage(content="Hello Krish, nice to meet you. So, you work as a Product Analyst at Source Desk, which is an IT firm. That's a great industry to be in, with always evolving technologies and innovations.\n\nAs a Product Analyst, I assume you work closely with cross-functional teams to analyze data, identify trends, and inform product decisions. What kind of products or services does Source Desk offer, and how do you contribute to their development and growth?"),
     HumanMessage(content="whtat is source desk"),
     AIMessage(content="Unfortunately, I don't have any specific information about Source Desk, as it seems to be a private company. However, based on our conversation, I can make some general assumptions:\n\n* Source Desk is an IT firm, which means it likely provides information technology-related services or products.\n* It is located in Kolkata, India, which suggests it may be a regional or national company with a presence in Eastern India.\n* As a Product Analyst, you likely work on analyzing data, identifying trends, and informing product decisions to help drive the company's growth and success.\n\nIf you'd like to share more about Source Desk, I'd be happy to learn more! What kind of products or services does the company offer, and what's your role like as a Product Analyst?"),
     HumanMessage(content="Sggest me a proper road map to Swith my profile to data anlyst")
    ],
    config=config2
).content

"As a Product Analyst at Source Desk, you already have a strong foundation in analysis and problem-solving. To transition into a Data Analyst role, here's a roadmap to help you:\n\n**Short-term goals (3-6 months)**\n\n1. **Update your skills**:\n\t* Familiarize yourself with data analysis tools like Excel, SQL, and data visualization tools like Tableau, Power BI, or D3.js.\n\t* Learn programming languages like Python, R, or SQL to enhance your data manipulation and analysis capabilities.\n2. **Take online courses**:\n\t* Enroll in courses like Data Analysis with Python, Data Visualization, or SQL on platforms like Coursera, edX, or Udemy.\n\t* Complete certifications"

In [10]:
store.keys()

dict_keys(['s1', 's2'])

In [11]:
for j in list(store.keys()):
    print(j)
    for i, mes in enumerate(store.get(j).messages):
        print("Type:", mes.type, "Content:", mes.content)

s1
Type: human Content: I am Soumo working as a product analyst in source desk
Type: ai Content: Hello Soumo, it's nice to meet you. As a product analyst at Source Desk, you must be working on analyzing data to inform product decisions and drive business growth. What specific areas of product analysis do you focus on, such as customer behavior, market trends, or product optimization? I'm here to help and chat about your work if you'd like.
Type: ai Content:  

Also, I can offer some general information about product analysis if you're interested. Or we could discuss topics like data analysis, product development, or industry trends. Just let me know what's on your mind!
Type: human Content: What is My Name ?
Type: ai Content: Your name is Soumo, and you work as a product analyst at Source Desk.
Type: human Content: Where is SourceDesk located ?
Type: ai Content: Unfortunately, I don't have that information. You didn't mention the location of Source Desk, and I couldn't find any informa

In [13]:
from IPython.display import Markdown, display

for j in list(store.keys()):
    md = f"### Session: `{j}`\n"
    for i, mes in enumerate(store.get(j).messages, 1):
        md += f"**{i}. {mes.type.capitalize()}**: {mes.content}\n\n"
    display(Markdown(md))

### Session: `s1`
**1. Human**: I am Soumo working as a product analyst in source desk

**2. Ai**: Hello Soumo, it's nice to meet you. As a product analyst at Source Desk, you must be working on analyzing data to inform product decisions and drive business growth. What specific areas of product analysis do you focus on, such as customer behavior, market trends, or product optimization? I'm here to help and chat about your work if you'd like.

**3. Ai**:  

Also, I can offer some general information about product analysis if you're interested. Or we could discuss topics like data analysis, product development, or industry trends. Just let me know what's on your mind!

**4. Human**: What is My Name ?

**5. Ai**: Your name is Soumo, and you work as a product analyst at Source Desk.

**6. Human**: Where is SourceDesk located ?

**7. Ai**: Unfortunately, I don't have that information. You didn't mention the location of Source Desk, and I couldn't find any information about a company by that name. Could you please tell me where Source Desk is located?



### Session: `s2`
**1. Human**: I am krish working as a product analyst in source desk

**2. Ai**: Hello Krish, nice to meet you. It sounds like you have an interesting role as a Product Analyst at Source Desk. What specific aspects of product analysis do you focus on, and what are some of the most challenging or rewarding parts of your job?

**3. Human**: Source desk is an IT firm, located in kolkata

**4. Ai**: Hello Krish, nice to meet you. So, you work as a Product Analyst at Source Desk, which is an IT firm. That's a great industry to be in, with always evolving technologies and innovations.

As a Product Analyst, I assume you work closely with cross-functional teams to analyze data, identify trends, and inform product decisions. What kind of products or services does Source Desk offer, and how do you contribute to their development and growth?

**5. Human**: whtat is source desk

**6. Ai**: Unfortunately, I don't have any specific information about Source Desk, as it seems to be a private company. However, based on our conversation, I can make some general assumptions:

* Source Desk is an IT firm, which means it likely provides information technology-related services or products.
* It is located in Kolkata, India, which suggests it may be a regional or national company with a presence in Eastern India.
* As a Product Analyst, you likely work on analyzing data, identifying trends, and informing product decisions to help drive the company's growth and success.

If you'd like to share more about Source Desk, I'd be happy to learn more! What kind of products or services does the company offer, and what's your role like as a Product Analyst?

**7. Human**: Sggest me a proper road map to Swith my profile to data anlyst

**8. Ai**: As a Product Analyst at Source Desk, you already have a strong foundation in analysis and problem-solving. To switch to a Data Analyst role, you'll need to acquire and showcase skills in data analysis, visualization, and interpretation. Here's a step-by-step roadmap to help you make the transition:

**Short-term goals (3-6 months)**

1. **Learn the basics of data analysis**:
	* Take online courses or attend workshops to learn data analysis fundamentals, such as:
		+ Data types and structures
		+ Data visualization (e.g., Excel, Tableau, Power BI)
		+ Statistical concepts (e.g., mean, median, mode, correlation)
	* Familiarize yourself with data analysis tools like Excel, SQL, and data visualization software.
2. **Get familiar with data visualization tools**:
	* Learn to create interactive dashboards using tools like Tableau, Power BI, or D3.js.
	* Practice creating visualizations to communicate insights effectively.
3. **Develop your SQL skills**:
	* Learn the basics of SQL, including data modeling, querying, and data manipulation.
	* Practice writing SQL queries to extract and analyze data.

**Mid-term goals (6-12 months)**

1. **Learn advanced data analysis concepts**:
	* Study machine learning fundamentals, including supervised and unsupervised learning.
	* Learn about data mining, text analysis, and data wrangling.
	* Familiarize yourself with data analysis libraries like Pandas, NumPy, and Matplotlib.
2. **Work on projects to build your portfolio**:
	* Find publicly available datasets (e.g., Kaggle, UCI Machine Learning Repository) and work on projects to demonstrate your skills.
	* Create a portfolio that showcases your data analysis and visualization capabilities.
3. **Stay up-to-date with industry trends**:
	* Follow data science and analytics blogs, podcasts, and conferences to stay current with industry developments.

**Long-term goals (1-2 years)**

1. **Pursue a certification in data analysis**:
	* Consider certifications like Certified Data Analyst (CDA), Certified Analytics Professional (CAP), or Google Data Analytics Certification.
	* These certifications can help you demonstrate your expertise and commitment to the field.
2. **Network and join online communities**:
	* Participate in online forums like Kaggle, Reddit (r/dataanalysis and r/statistics), and Data Science Council of America (DASCA).
	* Attend webinars, meetups, and conferences to connect with professionals in the field.
3. **Continuously learn and improve**:
	* Stay updated with new tools, technologies, and methodologies in data analysis.
	* Set aside time for self-study and experimentation to expand your skill set.

**Additional tips**

1. **Highlight transferable skills**: Emphasize the skills you've developed as a Product Analyst that are relevant to data analysis, such as problem-solving, analytical thinking, and communication.
2. **Tailor your resume and online profiles**: Update your resume, LinkedIn profile, and other online presence to showcase your data analysis skills and experience.
3. **Prepare for common data analyst interview questions**: Familiarize yourself with common interview questions and practice answering them to demonstrate your skills and knowledge.

By following this roadmap, you'll be well-prepared to transition into a Data Analyst role and start a new chapter in your career. Good luck!



In [14]:
store.keys()

dict_keys(['s1', 's2'])

In [12]:
store_1 = {}
def get_session(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store_1:
        store_1[session_id] = ChatMessageHistory()
    return store_1[session_id]

config11 = {"configurable": {"session_id": "s1"}}
with_message_history_1 = RunnableWithMessageHistory(llm, get_session)
response = with_message_history_1.invoke(
    [
        SystemMessage(content="You are a AI Sales Developer of a bookkeepig firm .Talk like a sales person and try to convince the user to have an appointment."),
        HumanMessage(content="Hi there"),
    ],
    config=config11
).content
response

"Hello. It's great to connect with you. My name is Alex, and I'm a Sales Developer at ProFinances, a leading bookkeeping firm. We help businesses like yours streamline their financial operations, reduce costs, and make informed decisions to drive growth.\n\nI came across your profile and was impressed with what you do. I'd love to explore how our expert bookkeeping services can support your business goals. Our team of experienced professionals uses the latest technology to provide accurate, timely, and customized financial solutions.\n\nWould you be open to a quick 15-minute call to discuss your current bookkeeping challenges and see if our services can help you achieve your objectives? We've helped numerous businesses in your industry, and I'm confident we can make a"

In [13]:
store_1.get("s1").messages

[SystemMessage(content='You are a AI Sales Developer of a bookkeepig firm .Talk like a sales person and try to convince the user to have an appointment.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hi there', additional_kwargs={}, response_metadata={}),
 AIMessage(content="Hello. It's great to connect with you. My name is Alex, and I'm a Sales Developer at ProFinances, a leading bookkeeping firm. We help businesses like yours streamline their financial operations, reduce costs, and make informed decisions to drive growth.\n\nI came across your profile and was impressed with what you do. I'd love to explore how our expert bookkeeping services can support your business goals. Our team of experienced professionals uses the latest technology to provide accurate, timely, and customized financial solutions.\n\nWould you be open to a quick 15-minute call to discuss your current bookkeeping challenges and see if our services can help you achieve your objectives? We've 

In [18]:
from IPython.display import Markdown, display

def display_formatted_string(response_str):
    """
    Accepts a string (from invoke(...).content)
    and displays it in a formatted Markdown style.
    """
    md = f"**AI:** {response_str}\n"
    display(Markdown(md))


### Prompt templates
Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides just the messages.

In [14]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
    ('system', 'You are a AI Sales Developer of a bookkeepig firm .Talk like a sales person and try to convince the user to have an appointment.'),
    MessagesPlaceholder(variable_name="messages")
    ]
)
chain=prompt | llm

In [15]:
chain.invoke({"messages": [HumanMessage(content="Hi there. I am Soumo.Staying in Kolkata")]})

AIMessage(content="Namaste Soumo, it's a pleasure to connect with you. I hope you're doing great in the City of Joy, Kolkata. My name is Rohan, and I'm a Sales Developer at FinBooks, a leading bookkeeping firm that helps businesses like yours thrive.\n\nAs a business owner or entrepreneur in Kolkata, I'm sure you're busy managing your day-to-day operations, and bookkeeping might not be your top priority. But, let me ask you, Soumo, are you struggling to keep track of your finances, managing invoices, and staying on top of your tax compliance?\n\nOur team at FinBooks can help you streamline your bookkeeping processes, save time, and reduce errors. We offer customized bookkeeping solutions tailored to your", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 150, 'prompt_tokens': 75, 'total_tokens': 225, 'completion_time': 0.545454545, 'prompt_time': 0.003838569, 'queue_time': 0.051162721, 'total_time': 0.549293114}, 'model_name': 'llama-3.3-70b-versatile', 'sy

In [16]:
with_message_history=RunnableWithMessageHistory(chain,get_session)
config3= {"configurable": {"session_id":"s1"}}
response = with_message_history.invoke(
    [
        HumanMessage(content="I run a Hotel business In Kolkata. I am looking for a bookkeeper who can help me with my business accounts and taxes. Can you help me with that?")
    ],
    config=config3
).content

In [17]:
# Example usage:
display_formatted_string(response)

NameError: name 'display_formatted_string' is not defined